# Module 5: Combining A2A and MCP

**Duration:** ~25 minutes

**Learning Objectives:**
- Understand when to use A2A vs MCP vs both
- Build agents that leverage both protocols
- Design hybrid architectures for complex systems
- Implement real-world patterns combining A2A and MCP

---

## 5.1 A2A + MCP: The Complete Picture

### Protocol Synergy

A2A and MCP are **complementary protocols** that together provide a complete solution for agent systems:

```
┌─────────────────────────────────────────────────────────────────────┐
│              The Complete Agent Communication Stack                  │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│                         ┌──────────────┐                            │
│                         │  Your Agent  │                            │
│                         └──────┬───────┘                            │
│                                │                                     │
│              ┌─────────────────┼─────────────────┐                  │
│              │                 │                 │                  │
│              ▼                 │                 ▼                  │
│     ┌────────────────┐        │        ┌────────────────┐          │
│     │      MCP       │        │        │      A2A       │          │
│     │  (Agent→Tool)  │        │        │ (Agent→Agent)  │          │
│     └───────┬────────┘        │        └───────┬────────┘          │
│             │                 │                │                    │
│     ┌───────┴───────┐        │        ┌───────┴───────┐            │
│     │               │        │        │               │            │
│     ▼               ▼        │        ▼               ▼            │
│ ┌────────┐    ┌─────────┐   │   ┌─────────┐    ┌─────────┐        │
│ │Database│    │   API   │   │   │ Travel  │    │ Finance │        │
│ │ Server │    │ Server  │   │   │  Agent  │    │  Agent  │        │
│ └────────┘    └─────────┘   │   └─────────┘    └─────────┘        │
│                             │         │               │            │
│                             │         │     (These agents also     │
│                             │         │      use MCP for tools)    │
│                             │         ▼               ▼            │
│                             │    ┌────────┐    ┌─────────┐        │
│                             │    │Booking │    │ Payment │        │
│                             │    │  API   │    │   API   │        │
│                             │    └────────┘    └─────────┘        │
│                             │                                      │
└─────────────────────────────┼──────────────────────────────────────┘
                              │
                   Your agent can use BOTH!
```

### Decision Framework

| Question | Use MCP | Use A2A |
|----------|---------|--------|
| Does it need to **reason**? | No | Yes |
| Is it a **function/API**? | Yes | No |
| Does it need **multi-turn**? | No | Yes |
| Is it **autonomous**? | No | Yes |
| Owned by **different org**? | Maybe | Usually |
| Needs **language support**? | Sometimes | Yes |

In [ ]:
# Setup
import os
from getpass import getpass

if 'GOOGLE_API_KEY' not in os.environ:
    os.environ['GOOGLE_API_KEY'] = getpass("Enter your Gemini API Key: ")

from google.adk.agents import Agent
from google.adk.tools import FunctionTool
from google.adk.runners import InMemoryRunner
from typing import Dict, Any, List
import random
import httpx

print("✅ Setup complete!")

## 5.2 Pattern 1: Agent with MCP Tools Exposed via A2A

This is the most common pattern: an agent uses MCP to access tools, and is exposed to other agents via A2A.

```
                              A2A
┌──────────────┐  ◄──────────────────────►  ┌──────────────────┐
│ Client Agent │                            │  Provider Agent  │
└──────────────┘                            │                  │
                                            │     ┌─────────┐  │
                                            │     │  MCP    │  │
                                            │     │ Toolset │  │
                                            │     └────┬────┘  │
                                            │          │       │
                                            └──────────┼───────┘
                                                       │ MCP
                                                       ▼
                                               ┌──────────────┐
                                               │   Database   │
                                               │   (BigQuery) │
                                               └──────────────┘
```

In [ ]:
# Simulating an Inventory Agent that uses MCP for database access
# and can be accessed by other agents via A2A

# Simulated MCP-style database tools
def query_inventory(product_name: str = None, category: str = None) -> Dict[str, Any]:
    """
    Query product inventory from the database.
    This would use MCP to connect to BigQuery in production.
    
    Args:
        product_name: Filter by product name (partial match)
        category: Filter by category
    
    Returns:
        Inventory data
    """
    # Simulated inventory database
    inventory = [
        {"id": "P001", "name": "Laptop Pro 15", "category": "Electronics", "stock": 45, "price": 1299.99},
        {"id": "P002", "name": "Wireless Mouse", "category": "Electronics", "stock": 230, "price": 29.99},
        {"id": "P003", "name": "Office Chair", "category": "Furniture", "stock": 18, "price": 349.99},
        {"id": "P004", "name": "Standing Desk", "category": "Furniture", "stock": 12, "price": 599.99},
        {"id": "P005", "name": "USB-C Hub", "category": "Electronics", "stock": 89, "price": 49.99},
        {"id": "P006", "name": "Monitor 27\" 4K", "category": "Electronics", "stock": 34, "price": 449.99},
    ]
    
    results = inventory
    
    if product_name:
        results = [p for p in results if product_name.lower() in p["name"].lower()]
    
    if category:
        results = [p for p in results if p["category"].lower() == category.lower()]
    
    return {
        "status": "success",
        "count": len(results),
        "products": results
    }


def update_inventory(product_id: str, quantity_change: int) -> Dict[str, Any]:
    """
    Update inventory quantity for a product.
    
    Args:
        product_id: The product ID to update
        quantity_change: Positive to add, negative to remove
    
    Returns:
        Update confirmation
    """
    # Simulated update
    return {
        "status": "success",
        "product_id": product_id,
        "quantity_change": quantity_change,
        "message": f"Inventory updated by {quantity_change:+d} units"
    }


def check_low_stock(threshold: int = 20) -> Dict[str, Any]:
    """
    Find products with stock below threshold.
    
    Args:
        threshold: Stock level threshold
    
    Returns:
        Low stock products
    """
    inventory = query_inventory()["products"]
    low_stock = [p for p in inventory if p["stock"] < threshold]
    
    return {
        "status": "success",
        "threshold": threshold,
        "count": len(low_stock),
        "low_stock_products": low_stock
    }


# Create Inventory Agent (would be exposed via A2A)
inventory_agent = Agent(
    name="inventory_manager",
    model="gemini-2.0-flash",
    description="Manages product inventory. Delegate inventory queries, stock checks, and updates here.",
    instruction="""
    You are an inventory management specialist with access to the product database.
    
    Capabilities:
    - Query inventory by product name or category
    - Check for low stock items
    - Update inventory quantities
    
    Always provide accurate inventory information and flag any low stock concerns.
    """,
    tools=[
        FunctionTool(query_inventory),
        FunctionTool(update_inventory),
        FunctionTool(check_low_stock)
    ]
)

print("✅ Inventory Agent created (simulating MCP database access)")

In [ ]:
# Test the inventory agent

async def chat_with_agent(agent: Agent, message: str) -> str:
    runner = InMemoryRunner(agent=agent)
    session = await runner.session_service.create_session(
        app_name=agent.name,
        user_id="workshop_user"
    )
    
    response_text = ""
    response = await runner.run(
        user_id="workshop_user",
        session_id=session.id,
        new_message=message
    )
    
    async for event in response:
        if hasattr(event, 'content') and event.content:
            for part in event.content.parts:
                if hasattr(part, 'text'):
                    response_text += part.text
    
    return response_text


response = await chat_with_agent(
    inventory_agent,
    "What electronics products do we have in stock? Are any running low?"
)

print("📦 Inventory Agent Test")
print("=" * 60)
print(response)

## 5.3 Pattern 2: Multi-Agent System with Shared MCP Resources

Multiple agents (communicating via A2A) can share access to common MCP resources:

```
┌─────────────┐        A2A        ┌─────────────┐
│   Sales     │◄────────────────►│  Inventory  │
│   Agent     │                   │   Agent     │
└──────┬──────┘                   └──────┬──────┘
       │                                 │
       │ MCP                             │ MCP
       │                                 │
       │      ┌─────────────────┐       │
       └─────►│   Shared MCP    │◄──────┘
              │    Database     │
              │    Server       │
              └─────────────────┘
```

In [ ]:
# Sales Agent - would communicate with Inventory Agent via A2A
# and access customer data via MCP

def search_customers(query: str = None) -> Dict[str, Any]:
    """Search customer database via MCP."""
    customers = [
        {"id": "C001", "name": "Acme Corp", "tier": "Enterprise", "lifetime_value": 125000},
        {"id": "C002", "name": "TechStart Inc", "tier": "SMB", "lifetime_value": 15000},
        {"id": "C003", "name": "Global Industries", "tier": "Enterprise", "lifetime_value": 450000},
    ]
    
    if query:
        customers = [c for c in customers if query.lower() in c["name"].lower()]
    
    return {"status": "success", "customers": customers}


def create_order(customer_id: str, products: List[Dict[str, Any]]) -> Dict[str, Any]:
    """Create a new order."""
    order_id = f"ORD-{random.randint(10000, 99999)}"
    total = sum(p.get("price", 0) * p.get("quantity", 1) for p in products)
    
    return {
        "status": "success",
        "order_id": order_id,
        "customer_id": customer_id,
        "products": products,
        "total": round(total, 2),
        "message": f"Order {order_id} created successfully"
    }


sales_agent = Agent(
    name="sales_agent",
    model="gemini-2.0-flash",
    description="Handles sales operations, customer queries, and order creation.",
    instruction="""
    You are a sales specialist. You can:
    - Search and retrieve customer information
    - Create orders for customers
    - Check inventory via the inventory_manager agent
    
    Before creating orders, always verify product availability with inventory.
    """,
    tools=[
        FunctionTool(search_customers),
        FunctionTool(create_order)
    ],
    sub_agents=[inventory_agent]  # A2A communication to inventory
)

print("✅ Sales Agent created with Inventory Agent as sub-agent")

In [ ]:
# Test the integrated system

response = await chat_with_agent(
    sales_agent,
    """I need to place an order for Acme Corp. They want:
    - 5 Laptop Pro 15s
    - 10 Wireless Mice
    
    Please check if we have enough in stock first."""
)

print("💼 Sales + Inventory Integration Test")
print("=" * 60)
print(response)

## 5.4 Pattern 3: Enterprise Architecture

A complete enterprise system combining both protocols:

```
┌──────────────────────────────────────────────────────────────────────┐
│                    Enterprise Agent Architecture                      │
├──────────────────────────────────────────────────────────────────────┤
│                                                                       │
│                         ┌─────────────────┐                          │
│                         │  Orchestrator   │                          │
│                         │     Agent       │                          │
│                         └────────┬────────┘                          │
│                                  │                                    │
│              ┌───────────────────┼───────────────────┐               │
│              │ A2A               │ A2A               │ A2A           │
│              ▼                   ▼                   ▼               │
│     ┌────────────────┐  ┌────────────────┐  ┌────────────────┐      │
│     │   Sales Agent  │  │  Inventory     │  │   Supplier     │      │
│     │                │  │    Agent       │  │    Agent       │      │
│     │  ┌──────────┐  │  │  ┌──────────┐  │  │  ┌──────────┐  │      │
│     │  │MCP: CRM  │  │  │  │MCP: DB   │  │  │  │MCP: API  │  │      │
│     │  └──────────┘  │  │  └──────────┘  │  │  └──────────┘  │      │
│     └────────────────┘  └────────────────┘  └────────────────┘      │
│              │                   │                   │               │
│              │ MCP               │ MCP               │ MCP           │
│              ▼                   ▼                   ▼               │
│     ┌────────────────┐  ┌────────────────┐  ┌────────────────┐      │
│     │   Salesforce   │  │    BigQuery    │  │   Vendor APIs  │      │
│     └────────────────┘  └────────────────┘  └────────────────┘      │
│                                                                       │
└──────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Supplier Agent - connects to external vendor APIs via MCP

def get_supplier_catalog(supplier_id: str = None) -> Dict[str, Any]:
    """Get product catalog from suppliers via API."""
    catalogs = {
        "SUP001": {
            "name": "TechSupply Co",
            "products": [
                {"sku": "TS-LP15", "name": "Laptop Pro 15", "unit_cost": 899, "moq": 10},
                {"sku": "TS-WM01", "name": "Wireless Mouse", "unit_cost": 15, "moq": 50},
            ],
            "lead_time_days": 7
        },
        "SUP002": {
            "name": "Office Essentials",
            "products": [
                {"sku": "OE-CH01", "name": "Office Chair", "unit_cost": 220, "moq": 5},
                {"sku": "OE-DSK1", "name": "Standing Desk", "unit_cost": 380, "moq": 3},
            ],
            "lead_time_days": 14
        }
    }
    
    if supplier_id:
        return {"status": "success", "catalog": catalogs.get(supplier_id, {})}
    return {"status": "success", "catalogs": catalogs}


def create_purchase_order(
    supplier_id: str,
    items: List[Dict[str, Any]]
) -> Dict[str, Any]:
    """Create a purchase order with a supplier."""
    po_number = f"PO-{random.randint(10000, 99999)}"
    
    return {
        "status": "success",
        "po_number": po_number,
        "supplier_id": supplier_id,
        "items": items,
        "estimated_delivery": "7-14 business days",
        "message": f"Purchase order {po_number} submitted to supplier"
    }


supplier_agent = Agent(
    name="supplier_manager",
    model="gemini-2.0-flash",
    description="Manages supplier relationships, catalogs, and purchase orders.",
    instruction="""
    You are a supplier management specialist. You can:
    - View supplier product catalogs
    - Create purchase orders for restocking
    - Compare prices across suppliers
    
    Always consider lead times and minimum order quantities.
    """,
    tools=[
        FunctionTool(get_supplier_catalog),
        FunctionTool(create_purchase_order)
    ]
)

print("✅ Supplier Agent created")

In [ ]:
# Create the Orchestrator Agent

orchestrator = Agent(
    name="operations_orchestrator",
    model="gemini-2.0-flash",
    description="Main orchestrator for business operations",
    instruction="""
    You are the operations coordinator for the business. You have access to:
    
    1. sales_agent - For customer and order management
    2. inventory_manager - For stock levels and inventory updates  
    3. supplier_manager - For procurement and supplier management
    
    Coordinate between these specialists to handle complex business requests.
    
    For example:
    - If stock is low, coordinate with supplier to reorder
    - If a large order comes in, verify inventory and possibly trigger restocking
    - Provide comprehensive business insights by combining data from all agents
    """,
    sub_agents=[sales_agent, inventory_agent, supplier_agent]
)

print(f"✅ Orchestrator created with {len(orchestrator.sub_agents)} sub-agents:")
for agent in orchestrator.sub_agents:
    print(f"   • {agent.name}")

In [ ]:
# Test the complete enterprise system

response = await chat_with_agent(
    orchestrator,
    """We need a full inventory health check:
    1. Which products are running low on stock?
    2. What suppliers can we order from to restock?
    3. Create purchase orders for any items below 20 units."""
)

print("🏢 Enterprise Orchestrator Test")
print("=" * 60)
print(response)

## 5.5 Real-World Architecture: E-Commerce Platform

Let's design a complete e-commerce system combining A2A and MCP:

In [ ]:
# E-Commerce Platform Architecture

ecommerce_architecture = '''
┌─────────────────────────────────────────────────────────────────────────┐
│                     E-Commerce Agent Platform                            │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│  ┌──────────────────────────────────────────────────────────────────┐   │
│  │                        Frontend Agents                            │   │
│  │  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐               │   │
│  │  │  Chat Bot   │  │  Voice Bot  │  │  Email Bot  │               │   │
│  │  │   Agent     │  │   Agent     │  │   Agent     │               │   │
│  │  └──────┬──────┘  └──────┬──────┘  └──────┬──────┘               │   │
│  └─────────┼────────────────┼────────────────┼──────────────────────┘   │
│            │                │                │                          │
│            └────────────────┼────────────────┘                          │
│                             │ A2A                                       │
│                             ▼                                           │
│  ┌──────────────────────────────────────────────────────────────────┐   │
│  │                      Router/Orchestrator                          │   │
│  │                    (Intent Classification)                        │   │
│  └──────────────────────────┬───────────────────────────────────────┘   │
│                             │                                           │
│       ┌─────────────────────┼─────────────────────┐                    │
│       │                     │                     │                    │
│       ▼                     ▼                     ▼                    │
│  ┌──────────┐         ┌──────────┐         ┌──────────┐               │
│  │ Product  │◄──A2A──►│  Order   │◄──A2A──►│  Support │               │
│  │  Agent   │         │  Agent   │         │  Agent   │               │
│  │          │         │          │         │          │               │
│  │ MCP:     │         │ MCP:     │         │ MCP:     │               │
│  │ •Catalog │         │ •Orders  │         │ •Tickets │               │
│  │ •Search  │         │ •Payment │         │ •FAQ     │               │
│  │ •Reviews │         │ •Shipping│         │ •Policy  │               │
│  └────┬─────┘         └────┬─────┘         └────┬─────┘               │
│       │                    │                    │                      │
│       │ MCP                │ MCP                │ MCP                  │
│       ▼                    ▼                    ▼                      │
│  ┌──────────┐         ┌──────────┐         ┌──────────┐               │
│  │ Product  │         │  Order   │         │  Support │               │
│  │   DB     │         │   DB     │         │    DB    │               │
│  │(BigQuery)│         │(Spanner) │         │(Firestore│               │
│  └──────────┘         └──────────┘         └──────────┘               │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
'''

print(ecommerce_architecture)

In [ ]:
# Implementation outline for the E-Commerce platform

ecommerce_code_template = '''
# agents/product_agent.py
from google.adk.agents import Agent
from google.adk.tools.mcp_tool import McpToolset, SseServerParameters

product_agent = Agent(
    name="product_agent",
    model="gemini-2.0-flash",
    description="Product search, recommendations, and reviews",
    instruction="Help customers find and learn about products.",
    tools=[
        McpToolset(
            connection_params=SseServerParameters(
                url="http://product-mcp:3000/mcp"
            )
        )
    ]
)

# agents/order_agent.py  
order_agent = Agent(
    name="order_agent",
    model="gemini-2.0-flash",
    description="Order placement, tracking, and returns",
    instruction="Help customers with orders and shipping.",
    tools=[
        McpToolset(
            connection_params=SseServerParameters(
                url="http://order-mcp:3000/mcp"
            )
        )
    ]
)

# agents/orchestrator.py
from google.adk.agents import RemoteA2aAgent

# Connect to agents via A2A (they could be in different services)
product_a2a = RemoteA2aAgent(
    name="product_service",
    agent_card="http://product-service/.well-known/agent.json"
)

order_a2a = RemoteA2aAgent(
    name="order_service",
    agent_card="http://order-service/.well-known/agent.json"
)

orchestrator = Agent(
    name="customer_assistant",
    model="gemini-2.0-flash",
    instruction="Route customer requests to appropriate specialists.",
    sub_agents=[product_a2a, order_a2a]
)
'''

print("📝 E-Commerce Implementation Template:")
print("=" * 60)
print(ecommerce_code_template)

## 5.6 Best Practices for Combined A2A + MCP

### Architecture Guidelines

| Principle | Description |
|-----------|-------------|
| **Separation of Concerns** | Each agent should have a single domain |
| **MCP at the Edge** | Use MCP for external system connections |
| **A2A for Coordination** | Use A2A for agent-to-agent communication |
| **Shared Nothing** | Agents shouldn't share state directly |
| **Idempotent Operations** | Tools should be safe to retry |

### Error Handling Strategy

```python
# Handle both MCP and A2A errors gracefully

async def robust_agent_call(agent, message, retries=3):
    for attempt in range(retries):
        try:
            response = await agent.run(message)
            return response
        except MCPConnectionError:
            # MCP server unreachable - maybe use cached data
            logger.warning(f"MCP connection failed, attempt {attempt+1}")
        except A2ATimeoutError:
            # Remote agent taking too long
            logger.warning(f"A2A timeout, attempt {attempt+1}")
        except Exception as e:
            logger.error(f"Unexpected error: {e}")
            break
    
    return fallback_response()
```

## 5.7 Exercise: Design a Healthcare System

Design a multi-agent healthcare system using both A2A and MCP:

**Requirements:**
1. Patient Agent: Answers patient questions, schedules appointments
2. Records Agent: Accesses medical records (HIPAA compliant)
3. Scheduling Agent: Manages doctor availability
4. Pharmacy Agent: Handles prescriptions and refills

In [ ]:
# Exercise: Design the healthcare architecture

# TODO: Create Agent Cards for each agent
# TODO: Identify which connections should be MCP vs A2A
# TODO: Define the tool interfaces for each MCP server

healthcare_design = '''
Your architecture here:

Agents:
1. Patient Agent
   - MCP Tools: ???
   - A2A Connections: ???

2. Records Agent
   - MCP Tools: ???
   - A2A Connections: ???

3. Scheduling Agent
   - MCP Tools: ???
   - A2A Connections: ???

4. Pharmacy Agent
   - MCP Tools: ???
   - A2A Connections: ???
'''

print(healthcare_design)

---

## Summary

In this module, you learned:

1. **Protocol Synergy**: A2A and MCP complement each other
2. **Decision Framework**: When to use MCP vs A2A
3. **Common Patterns**: Agent+MCP exposed via A2A
4. **Enterprise Architecture**: Multi-agent systems with shared resources
5. **Best Practices**: Separation of concerns, error handling

### Key Takeaways

- Use **MCP** for connecting to tools, databases, and APIs
- Use **A2A** for agent collaboration and delegation
- Agents can use **both** protocols simultaneously
- Design for **loose coupling** between agents

### Next Steps

Continue to **Module 6: Vertex AI Deployment** to learn how to deploy these systems to production.

---

### Architecture Checklist

- [ ] Each agent has a single responsibility
- [ ] MCP used for external system access
- [ ] A2A used for inter-agent communication
- [ ] Error handling at protocol boundaries
- [ ] Observability and logging configured
- [ ] Security considerations addressed